In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import logging
import pandas as pd
from rdkit import Chem
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from atom2024 import utils
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
pd.set_option('display.max_colwidth', None)

In [25]:
suppl = Chem.SDMolSupplier("data/cdk2.sdf")

pmol = []

for mol in suppl:
    pmol.append(utils.normalize(mol))


[18:27:43] Initializing MetalDisconnector
[18:27:43] Running MetalDisconnector
[18:27:43] Initializing Normalizer
[18:27:43] Running Normalizer
[18:27:43] Running LargestFragmentChooser
[18:27:43] Fragment: CC(C)C(=O)COc1nc(N)nc2[nH]cnc12
[18:27:43] New largest fragment: CC(C)C(=O)COc1nc(N)nc2[nH]cnc12 (30)
[18:27:43] Running Uncharger
[18:27:43] Initializing MetalDisconnector
[18:27:43] Running MetalDisconnector
[18:27:43] Initializing Normalizer
[18:27:43] Running Normalizer
[18:27:44] Initializing MetalDisconnector
[18:27:44] Running MetalDisconnector
[18:27:44] Initializing Normalizer
[18:27:44] Running Normalizer
[18:27:44] Running LargestFragmentChooser
[18:27:44] Fragment: Nc1nc(OC[C@@H]2CCCO2)c2nc[nH]c2n1
[18:27:44] New largest fragment: Nc1nc(OC[C@@H]2CCCO2)c2nc[nH]c2n1 (30)
[18:27:44] Running Uncharger
[18:27:44] Initializing MetalDisconnector
[18:27:44] Running MetalDisconnector
[18:27:44] Initializing Normalizer
[18:27:44] Running Normalizer
[18:27:44] Initializing MetalDis